In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Evaluation on Test Set\n",
    "Evaluate trained agent on 2000 test games and compute final metrics."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import pickle\n",
    "import json\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from tqdm.notebook import tqdm\n",
    "\n",
    "sys.path.append('..')\n",
    "\n",
    "from src.env.hangman_env import HangmanEnv\n",
    "from src.hmm.oracle import HMMOracle\n",
    "from src.hmm.emissions import EmissionBuilder\n",
    "from src.rl.q_learning import QLearningAgent\n",
    "from src.utils.data_loader import load_test_words\n",
    "from src.utils.metrics import MetricsCalculator"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Test Data and Models"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load test words\n",
    "all_test_words = load_test_words('../data/raw/text.txt')\n",
    "test_words = all_test_words[:2000]  # Use first 2000\n",
    "\n",
    "print(f\"Loaded {len(test_words)} test words\")\n",
    "\n",
    "# Load preprocessed data\n",
    "with open('../data/processed/words_by_length.pkl', 'rb') as f:\n",
    "    words_by_length = pickle.load(f)\n",
    "\n",
    "# Load HMM\n",
    "emissions = EmissionBuilder.load('../models/hmm/emissions.pkl')\n",
    "oracle = HMMOracle(emissions, words_by_length)\n",
    "\n",
    "# Load agent\n",
    "agent = QLearningAgent()\n",
    "agent.load('../models/rl/q_table_final.pkl')\n",
    "agent.epsilon = 0.0  # No exploration\n",
    "\n",
    "print(\"Models loaded successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Run Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "metrics_calc = MetricsCalculator()\n",
    "\n",
    "for word in tqdm(test_words, desc=\"Evaluating\"):\n",
    "    env = HangmanEnv(word, max_lives=6)\n",
    "    state = env.reset()\n",
    "    done = False\n",
    "    \n",
    "    while not done:\n",
    "        hmm_probs = oracle.get_letter_probs(state['mask'], state['guessed_letters'])\n",
    "        valid_actions = env.get_valid_actions()\n",
    "        action = agent.select_action(state, hmm_probs, valid_actions, training=False)\n",
    "        state, reward, done, info = env.step(action)\n",
    "    \n",
    "    metrics_calc.add_game(\n",
    "        word=word,\n",
    "        won=info.get('win', False),\n",
    "        wrong_guesses=env.wrong_guesses,\n",
    "        repeated_guesses=env.repeated_guesses\n",
    "    )\n",
    "\n",
    "print(\"\\nEvaluation complete!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Compute and Display Metrics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "metrics = metrics_calc.compute_metrics()\n",
    "df = metrics_calc.get_dataframe()\n",
    "\n",
    "print(\"=\" * 60)\n",
    "print(\"EVALUATION RESULTS\")\n",
    "print(\"=\" * 60)\n",
    "print(f\"Total Games: {metrics['total_games']}\")\n",
    "print(f\"Wins: {metrics['wins']}\")\n",
    "print(f\"Losses: {metrics['losses']}\")\n",
    "print(f\"Success Rate: {metrics['success_rate']*100:.2f}%\")\n",
    "print(f\"Total Wrong Guesses: {metrics['total_wrong_guesses']}\")\n",
    "print(f\"Total Repeated Guesses: {metrics['total_repeated_guesses']}\")\n",
    "print(f\"Avg Wrong/Game: {metrics['avg_wrong_per_game']:.2f}\")\n",
    "print(f\"Avg Repeated/Game: {metrics['avg_repeated_per_game']:.2f}\")\n",
    "print(f\"\\nFINAL SCORE: {metrics['final_score']:.2f}\")\n",
    "print(\"=\" * 60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualize Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n",
    "\n",
    "# Win/Loss pie chart\n",
    "axes[0, 0].pie([metrics['wins'], metrics['losses']], \n",
    "               labels=['Wins', 'Losses'],\n",
    "               autopct='%1.1f%%',\n",
    "               colors=['#2ecc71', '#e74c3c'])\n",
    "axes[0, 0].set_title(f\"Success Rate: {metrics['success_rate']*100:.1f}%\")\n",
    "\n",
    "# Wrong guesses distribution\n",
    "axes[0, 1].hist(df['wrong_guesses'], bins=range(0, 8), edgecolor='black')\n",
    "axes[0, 1].set_xlabel('Wrong Guesses')\n",
    "axes[0, 1].set_ylabel('Frequency')\n",
    "axes[0, 1].set_title('Wrong Guesses Distribution')\n",
    "axes[0, 1].grid(axis='y', alpha=0.3)\n",
    "\n",
    "# Repeated guesses distribution\n",
    "if df['repeated_guesses'].max() > 0:\n",
    "    axes[1, 0].hist(df['repeated_guesses'], \n",
    "                   bins=range(0, df['repeated_guesses'].max()+2), \n",
    "                   edgecolor='black')\n",
    "else:\n",
    "    axes[1, 0].text(0.5, 0.5, 'No repeated guesses!', \n",
    "                   ha='center', va='center', fontsize=14)\n",
    "axes[1, 0].set_xlabel('Repeated Guesses')\n",
    "axes[1, 0].set_ylabel('Frequency')\n",
    "axes[1, 0].set_title('Repeated Guesses Distribution')\n",
    "axes[1, 0].grid(axis='y', alpha=0.3)\n",
    "\n",
    "# Performance by word length\n",
    "df['word_length'] = df['word'].apply(len)\n",
    "length_stats = df.groupby('word_length')['won'].agg(['sum', 'count'])\n",
    "length_stats['success_rate'] = length_stats['sum'] / length_stats['count']\n",
    "axes[1, 1].bar(length_stats.index, length_stats['success_rate'], edgecolor='black')\n",
    "axes[1, 1].set_xlabel('Word Length')\n",
    "axes[1, 1].set_ylabel('Success Rate')\n",
    "axes[1, 1].set_title('Success Rate by Word Length')\n",
    "axes[1, 1].grid(axis='y', alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Analyze Failures"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get failed words\n",
    "failed_words = df[df['won'] == False]['word'].values\n",
    "\n",
    "print(f\"\\nFailed words: {len(failed_words)}\")\n",
    "print(f\"\\nSample failed words (first 20):\")\n",
    "for word in failed_words[:20]:\n",
    "    print(word)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Save Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from pathlib import Path\n",
    "\n",
    "# Create output directory\n",
    "Path('../reports/results').mkdir(parents=True, exist_ok=True)\n",
    "\n",
    "# Save metrics\n",
    "with open('../reports/results/final_metrics.json', 'w') as f:\n",
    "    json.dump(metrics, f, indent=2)\n",
    "\n",
    "# Save detailed results\n",
    "df.to_csv('../reports/results/evaluation_results.csv', index=False)\n",
    "\n",
    "print(\"Results saved to reports/results/\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.12.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
